In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

!\cp "drive/My Drive/WaveGen3/config.py" .
!\cp "drive/My Drive/WaveGen3/model.py" .
!\cp "drive/My Drive/WaveGen3/data.py" .
!\cp "drive/My Drive/WaveGen3/train.py" .
!\cp "drive/My Drive/WaveGen3/interact.py" .
!\cp "drive/My Drive/WaveGen3/ext.py" .

!pip uninstall -y torch                                                       &> /dev/null
!pip install -q torch==1.5.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install "drive/My Drive/torch-1.5.0+cu101-cp37-cp37m-linux_x86_64.whl"   &> /dev/null
# !pip show torch
!pip install librosa --upgrade                                                &> /dev/null
# !pip uninstall -y librosa                                                   &> /dev/null
# !pip install librosa                                                        &> /dev/null
# !pip show librosa

In [ ]:
import config
config.model_path      = 'drive/My Drive/WaveGen3/models/model'
config.data_path       = 'drive/My Drive/WaveGen3/data'

config.dev_ratio       = 0
config.batch_size      = 10

config.state_size      = config.timestep_size
config.state_out_delta = True
config.hm_heads        = 1
config.init_xavier     = True

config.hm_epochs       = 100
config.learning_rate   = 1e-7
config.loss_squared    = True
config.max_seq_len     = 0
config.optimizer       = 'custom'
config.gradient_clip   = 0

config.model_path      = 'models/model'
config.fresh_model     = True
config.fresh_meta      = True
config.ckp_per_ep      = config.hm_epochs//10

config.use_gpu         = False

In [ ]:
# from data import main; main();
from train import main; main();

KeyboardInterrupt: ignored

In [ ]:
config.model_path     = f'drive/My Drive/WaveGen3/models/model_ckp{config.hm_epochs-1}'
config.output_file    = 'drive/My Drive/WaveGen3/resp'
config.hm_wav_gen     = 3
config.hm_extra_steps = 10

from interact import main; main();

In [ ]:
from data import load_data
from model import load_model, respond_to

d = load_data()
m = load_model()
respond_to(m,d)

for m in m:
  for l in m:
    for k,v in l._asdict().items():
      print(f'{k}, {v.requires_grad}, {(v.grad/len(d)).pow(2).sum()/(v.size(0)*v.size(1))}')